# GEMS PedTools Example Usage for R
Below is a simple example that illustrates how to access data in the PedTools database.
### Set up an HTTP client using Python's request library
We use a `Session` object to store our API key and automatically include it in the header for each request.

Note that we have a `api_key.R` file in the Exchange-Notebooks directory. The file contains only the below line.
```
api_key <- 'SECRET'
```

In [1]:
# Load necessary libraries
library(httr)
library(jsonlite)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




### Set up an HTTP client

In [1]:
# Grab your API Key and assign it to api_key
source("api_key.R")
#api_key <- "mgM3W0g58Xv3v6B1X16-Gg"

# Initialize session headers
headers <- add_headers(apikey = api_key)

# Define base URL
# Uncomment the base URL you wish to use
base <- "https://exchange-1.gems.msi.umn.edu/pedtools/v1"
# base <- "https://exchange-dev.gems.msi.umn.edu/pedtools/v1"


ERROR: Error in add_headers(apikey = api_key): could not find function "add_headers"


### Grab all information for a specific variety

In [3]:
# Query information for a specific variety
params <- list(pedigree_depth = 5)
variety <- URLencode("TURKEY", reserved = TRUE)
url <- paste0(base, "/", variety)

# Perform GET request
response <- GET(url, headers, query = params)

# Check if the request was successful
if (status_code(response) == 200) {
  # Parse JSON response and convert to a data frame
  content <- content(response, as = "text", encoding = "UTF-8")
  data <- fromJSON(content, flatten = TRUE) %>% as.data.frame()
  print(data)

  # Example: Checking aliases for returned varieties
  if ("aliases" %in% colnames(data)) {
    print("Aliases for the varieties:")
    print(data$aliases)
  } else {
    print("No aliases found.")
  }
} else {
  cat("Error:", status_code(response), "-", content(response, as = "text"), "\n")
}

    preferred_name crop_name is_landrace backcross_depth selfing_count
1   TURKEY GID:135     wheat       FALSE               0             2
2 TURKEY GID:10509     wheat       FALSE               0             0
  market_class release_date developer parentage
1           NA           NA        NA        NA
2           NA           NA        NA        NA
                                                                                                                                   aliases
1 135, CWI51783, P.1066-?-?, TK, TURKEY, germplasm_bank_id, germplasm_bank_accession_id, selection_history, cross_abbreviation, cross_name
2                                                                          10509, TURKEY, TURKEY 13.R, germplasm_bank_id, bcid, cross_name
  father              pedigree mother.preferred_name mother.crop_name
1     NA P.1066 GID:25 (25) F2     P.1066-? GID:3700            wheat
2     NA      TURKEY GID:10509                  <NA>             <NA>
  mother.is_land